In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\abhay\\Downloads\\Medical-Chatbot-Generative-AI'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
! pip install --upgrade sentence-transformers


  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [5]:
from pinecone import Pinecone

c:\Users\abhay\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
! pip install langchain-groq


In [7]:
# -----------------------
# 0. Import necessary modules
# -----------------------
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set your API Keys
GROQ_API_KEY = "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"
PINECONE_API_KEY = "pcsk_7L4dwj_5zyQBcw6KboyFBSGnEbfYhH79CCKcN6wYgfAAfTnVb1sHNdiuj3kVsSgDvvHafr"

# Set environment variables
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# -----------------------
# 1. Load and Split PDF Data
# -----------------------
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_file(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

# Load the PDFs
extracted_data = load_pdf_file(data_path="Data/")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))

# -----------------------
# 2. Download HuggingFace Embeddings
# -----------------------
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

query_result = embeddings.embed_query("Hello world")
print("Embedding vector length:", len(query_result))

# -----------------------
# 3. Set up Pinecone (LATEST 2025 METHOD)
# -----------------------
from pinecone import Pinecone, ServerlessSpec

# Connect to Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mkc"

# Create index if it doesn't exist
'''if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
'''

# -----------------------
# 4. Upload Embeddings to Pinecone
# -----------------------
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

# Or connect to existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# -----------------------
# 5. Set up Retriever
# -----------------------
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("What is Acne?")
print("Sample Retrieved Docs:", retrieved_docs)

# -----------------------
# 6. Set up Groq LLM (instead of OpenAI)
# -----------------------


FileNotFoundError: Directory not found: 'Data/'

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.4,
    max_tokens=500,
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY
)

# -----------------------
# 7. Create RAG Chain
# -----------------------
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# -----------------------
# 8. Test the RAG System
# -----------------------
response = rag_chain.invoke({"input":"Lately, I've been feeling extremely tired, even after a full night's sleep. I also noticed that my heart races sometimes and I get short of breath just from climbing a few stairs. I've had some swelling in my ankles too. I thought it was just stress, but it’s been going on for weeks now."})
print("Response 1:", response["answer"])



In [ ]:
response = rag_chain.invoke({"input": "I have asthma and sometimes struggle with shortness of breath. What treatments are recommended, and what precautions should I take to manage my condition"})
print("Response 1:", response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "i am suffering from coldaswell as high pain in body suggest me some   medicine "})
print("Response 1:", response["answer"])

In [ ]:
response = rag_chain.invoke({"input": " give me sybptom of maleria"})
print("Response 1:", response["answer"])